# Function-calling with an OpenAPI specification


Much of the internet is powered by RESTful APIs. Giving GPT the ability to call them opens up a world of possibilities. This notebook demonstrates how GPTs can be used to intelligently call APIs. It leverages OpenAPI specifications and chained function calls.

The [OpenAPI Specification (OAS)](https://swagger.io/specification/) is a universally accepted standard for describing the details of RESTful APIs in a format that machines can read and interpret. It enables both humans and computers to understand the capabilities of a service, and it can be leveraged to show GPT how to call APIs.

This notebook is divided into two main sections:

1. How to convert a sample OpenAPI specification into a list of function definitions for the chat completions API.
2. How to use the chat completions API to intelligently invoke these functions based on user instructions.

We recommend familiariazing yourself with [function-calling](./How_to_call_functions_with_chat_models.ipynb) before proceding.


In [ ]:
import os
import json
import jsonref
from openai import AzureOpenAI
import requests
from pprint import pp
import tiktoken
import os


client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
tikoken_encoding = tiktoken.get_encoding("cl100k_base")
DEPLOYMENT_NAME_GPT4O:str= os.getenv("DEPLOYMENT_NAME_GPT4O")
DEPLOYMENT_NAME_GPT35: str = os.getenv("DEPLOYMENT_NAME_GPT35")

## How to convert an OpenAPI specification into function definitions


The OpenAPI specification is extracted from https://petstore.swagger.io/, a dummy API for showing the Swagger capabilities, and the endpoints listed can be used without authentication. Check out the websites for which input parameters are supported.

Before we proceed, let's inspect OpenAPI specification. OpenAPI specs include details about the API's endpoints, the operations they support, the parameters they accept, the requests they can handle, and the responses they return. The spec is defined in JSON format.

Each operation in the spec has an `operationId`, which we will use as the function name when we parse the spec into function specifications. The spec also includes schemas that define the data types and structures of the parameters for each operation.

In [ ]:
with open('../data/openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read()) # it's important to load with jsonref, as explained below

display(openapi_spec)

Now that we have a good understanding of the OpenAPI spec, we can proceed to parse it into function specifications.

We can write a simple `openapi_to_functions` function to generate a list of definitions, where each function is represented as a dictionary containing the following keys:

- `name`: This corresponds to the operation identifier of the API endpoint as defined in the OpenAPI specification.
- `description`: This is a brief description or summary of the function, providing an overview of what the function does.
- `parameters`: This is a schema that defines the expected input parameters for the function. It provides information about the type of each parameter, whether it is required or optional, and other related details.

For each of the endpoints defined in the schema, we need to do the following:

1. **Resolve JSON references**: In an OpenAPI specification, it's common to use JSON references (also known as $ref) to avoid duplication. These references point to definitions that are used in multiple places. For example, if multiple API endpoints return the same object structure, that structure can be defined once and then referenced wherever it's needed. We need to resolve and replace these references with the content they point to.

2. **Extract a name for the functions:** We will simply use the operationId as the function name. Alternatively, we could use the endpoint path and operation as the function name.

3. **Extract a description and parameters:** We will iterate through the `description`, `summary`, `requestBody` and `parameters` fields to populate the function's description and parameters.

Here's the implementation:


In [ ]:
def openapi_to_functions(openapi_spec):
    functions = []

    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. Resolve JSON references.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. Extract a name for the functions.
            function_name = spec.get("operationId")

            # 3. Extract a description and parameters.
            desc = spec.get("description") or spec.get("summary", "")

            schema = {"type": "object", "properties": {}}

            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            functions.append(
                {"type": "function", "function": {"name": function_name, "description": desc, "parameters": schema}}
            )

    return functions


functions = openapi_to_functions(openapi_spec)

### Inspect the generated functions (OpenAI tools) that were generated from the OpenAPI spec 

In [ ]:
# Print the resulting functions
for function in functions:
    pp(function)
    print()

Since the functions needs to be passed to OpenAI for each query, we need to know how many tokens will be used. Here we can use the tiktoken libary

In [ ]:
# Total number of tokens for the functions.
len(tikoken_encoding.encode(str(functions)))

### Define methods

OpenAI will return the endpoints to use based on the user query, and we need to map this to actually methods that can execute these. These are defined below:

In [ ]:
def getPetById(id: int):
    return requests.get(f"https://petstore.swagger.io/v2/pet/{id}").text

def getInventory():
    return requests.get("https://petstore.swagger.io/v2/store/inventory").text

## How to call these functions with GPT


Now that we have these function definitions, we can leverage GPT to call them intelligently based on user inputs.

It's important to note that the chat completions API does not execute the function; instead, it generates the JSON that you can use to call the function in your own code.

For more information on function-calling, refer to our dedicated [function-calling guide](./How_to_call_functions_with_chat_models.ipynb).


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant.
Respond to the following prompt by using function_call and then summarize actions.
Ask for clarification if a user request is ambiguous.
"""

# Maximum number of function calls allowed to prevent infinite or lengthy loops
MAX_CALLS = 5


def get_openai_response(functions, messages):
    return client.chat.completions.create(
        model=DEPLOYMENT_NAME_GPT35,
        tools=functions,
        tool_choice="auto",  # "auto" means the model can pick between generating a message or calling a function.
        temperature=0,
        messages=messages,
    )


def process_user_instruction(functions, instruction):
    num_calls = 0
    messages = [
        {"content": SYSTEM_MESSAGE, "role": "system"},
        {"content": instruction, "role": "user"},
    ]

    while num_calls < MAX_CALLS:
        response = get_openai_response(functions, messages)
        message = response.choices[0].message
        
        try:
            if message.tool_calls is None:
                # No more tools required, assume done and printing last message
                pp("\n\n" + message)
                break

            print(f"\n>> Function call #: {num_calls + 1}\n")
            pp(message.tool_calls)
            messages.append(message)

            result: str

            # Here we extract information from Azure OpenAI about which method to use, and which parameters
            args = json.loads(message.tool_calls[0].function.arguments)
            if message.tool_calls[0].function.name == "getPetById":
                result = getPetById(args["parameters"]["id"])

            if message.tool_calls[0].function.name == "getInventory":
                result = getInventory()

            print("API result")
            pp(result)

            # Add the result from the API call, and send the results back Azure OpenAI
            messages.append(
                {
                    "role": "tool",
                    "content": result,
                    "tool_call_id": message.tool_calls[0].id,
                }
            )

            num_calls += 1
        except:
            print("\n>> Message:\n")
            print(message.content)
            break

    if num_calls >= MAX_CALLS:
        print(f"Reached max chained function calls: {MAX_CALLS}")


### Try out our system

In [ ]:
USER_INSTRUCTION = """
Instruction: 
What is the current inventory in the store, and is something empty?
"""

process_user_instruction(functions, USER_INSTRUCTION)

In [16]:
USER_INSTRUCTION = """
Instruction: 
What dog has id 3 and is it available?
"""

process_user_instruction(functions, USER_INSTRUCTION)


>> Function call #: 1

[ChatCompletionMessageToolCall(id='call_39rWauCOYeTrf8N6TIuI43Ng', function=Function(arguments='{"parameters":{"id":1}}', name='getPetById'), type='function')]
API result
'{"code":1,"type":"error","message":"Pet not found"}'

>> Message:

I'm sorry, but the dog with ID 1 was not found in the system. Can you please provide more information or verify the ID?
